In [1]:
import sys
# caution: path[0] is reserved for script path (or '' in REPL)
work_path = 'c:\\Users\\Edward TL\\OneDrive\\Peryton\\data_app'

sys.path.insert(1, work_path)


from helpers import get_config, get_excels

config = get_config(f"{work_path}/extract/cnt_files.env")
db_path = f"{work_path}\\{config['CNT_DB']}"
excels = get_excels(db_path)

In [2]:
excels

['Arma BHA.xlsx',
 'BITACORA_MAESTRA.xlsx',
 'Desarma BHA.xlsx',
 'Metros perforados por hora.xlsx',
 'Plantilla de reporte.xlsx',
 'Tiempo de conexion.xlsx',
 'Velocidad de viaje.xlsx']

In [3]:
excels.remove('Plantilla de reporte.xlsx')
excels.remove("BITACORA_MAESTRA.xlsx")

In [4]:
excels

['Arma BHA.xlsx',
 'Desarma BHA.xlsx',
 'Metros perforados por hora.xlsx',
 'Tiempo de conexion.xlsx',
 'Velocidad de viaje.xlsx']

In [5]:
import pandas as pd

df = {
    file.replace(".xlsx", ""): pd.read_excel(
            f"{db_path}\\{file}") for file in excels
}

df['Arma BHA']

,Estándar,Desarma BHA Tipo 6
0,3,2.55


In [6]:
from openpyxl import load_workbook
from openpyxl.workbook import Workbook

def create_wb(file_name: str = "Reporte de prueba 1.xlsx") -> None:
    """Creates a copy from an base file, and returns the new Workbook
    ready to work."""
    base_wb = load_workbook(f"{db_path}\\Plantilla de reporte.xlsx")
    new_file = f"{db_path}\\Reportes\\{file_name}"
    base_wb.save(new_file)
    base_wb.close()

    return 

create_wb()

In [7]:
test_report_file = f"{db_path}\\Reportes\\Reporte de prueba 1.xlsx"

In [8]:
def append_df_sheet(df, file, sheet_name) -> None:
    writer = pd.ExcelWriter(file, engine="openpyxl", mode = 'a')
    df.to_excel(
        excel_writer = writer,
        sheet_name = sheet_name,
        index = False
        )
    writer.close()

for table in df:
    append_df_sheet(df[table], test_report_file, table)

In [9]:
book = load_workbook(test_report_file)
print(book.sheetnames)

['Reporte', 'Arma BHA', 'Desarma BHA', 'Metros perforados por hora', 'Tiempo de conexion', 'Velocidad de viaje']


In [10]:
from reporter.plots import Bar
import reporter.plot.objects as po

In [11]:
def create_charts(charts_dictionary, h, w):
    for chart_name, chart_pos in charts_dictionary.items():
        chart = Bar(
            _data = po.ChartData(
                titles= po.Titles(
                    chart = chart_name,
                    y_axis = "Tiempo (h)",
                    x_axis = "Comparativa"
                ),
                chart = po.Source(
                    ws = book['Reporte'],
                    df = df[chart_name]
                ),
                data = po.Source(
                    ws = book[chart_name],
                    df = df[chart_name]
                ),
                ref = po.ChartReferences(
                    chart_position=chart_pos
                ),
                height = h,
                width = w
            )
        ).to_excel()

In [12]:
armed_chars = {
    "Arma BHA": "A3",
    "Desarma BHA": "N3"
}

create_charts(armed_chars, 4, 8)

long_charts = {
    'Tiempo de conexion': "A11",
    "Velocidad de viaje": "A22"
}

create_charts(long_charts, 6, 15)

book.save(test_report_file)
book.close()

In [13]:
df.keys()

dict_keys(['Arma BHA', 'Desarma BHA', 'Metros perforados por hora', 'Tiempo de conexion', 'Velocidad de viaje'])

In [14]:
df['Desarma BHA']

,Estándar,3.5
0,Desarma BHA Tipo 6,2.55
